# Load Libraries

In [27]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from keras.models import model_from_json
import pandas as pd

# Load Model

Model Structure:

In [9]:
# load json and create model
json_file = open('model/model_structure.json','r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

Model Weights:

In [10]:
# load weights into new model
model.load_weights("model/model_weights.h5")

Compile model:

In [11]:
# compile loaded model 
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Load Data

In [18]:
def load_dataset(prefix):
    # load data and labels
    X = load_dataset_group(prefix)  
   
    # return dataset
    return X

In [14]:
# load a dataset group
def load_dataset_group(group):
    
    # load all 9 files as a single array
    filenames = list()
    
    # total acceleration
    filenames += ['/Inertial Signals/total_acc_x.txt',
                  '/Inertial Signals/total_acc_y.txt',
                  '/Inertial Signals/total_acc_z.txt']
    
    # body acceleration
    filenames += ['/Inertial Signals/body_acc_x.txt',
                  '/Inertial Signals/body_acc_y.txt',
                  '/Inertial Signals/body_acc_z.txt']
    
    # body gyroscope
    filenames += ['/Inertial Signals/body_gyro_x.txt',
                  '/Inertial Signals/body_gyro_y.txt',
                  '/Inertial Signals/body_gyro_z.txt']
    
    # load input data
    X = load_group(filenames, group)
       
    # return X
    return X

In [15]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, group):
    loaded = list()
    
    for name in filenames:

        data = load_file(group+name)
        loaded.append(data)
    
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [16]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [17]:
# load data
X = load_dataset('data_predictions')

# Make Predictions

In [23]:
def make_prediction(X):
    # define parameters
    verbose = 0
    samples = X.shape[0]
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features   

    
    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    X = X.reshape((samples, time_steps, rows, columns, channels))
        
    # fit network
    class_predictions = model.predict_classes(X, verbose=verbose)    
    
    return class_predictions

In [30]:
predictions = pd.DataFrame(make_prediction(X))

In [31]:
predictions

,0
0,4
1,4
2,4
3,4
4,4
...,...
2942,1
2943,1
2944,1
2945,1


# Save Predictions

In [32]:
predictions.to_csv('data_predictions/predictions.csv')